<a href="https://colab.research.google.com/github/sheldonkemper/bank_of_england/blob/Preprocessing/notebooks/processed/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Library**

In [65]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("punkt_tab")
from nltk.tokenize import word_tokenize, sent_tokenize
from collections import Counter
import regex as re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [3]:
!pip install tensorflow
!pip install numpy
!pip install bertopic


import tensorflow as tf
import numpy as np
import random
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
import tensorflow as tf
from umap import UMAP
from bertopic.representation import MaximalMarginalRelevance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5

# **Function**

preprocessor function : if you add data name and column name, the values in the column are preprocessed.

In [4]:
"""
#create function to preprocess data
def preprocessor (data, col):
  #Lower the lettercase
  data[col] = data[col].str.lower()

  #Remove stop words
  stop_words = set(stopwords.words("english"))
  data[col] = data[col].apply(lambda x: " ".join([word for word in str(x).split() if word not in (stop_words)]))

  #Tokenize the word
  data[col] = data[col].apply(word_tokenize)

  #Remove numbers
  data[col] = data[col].apply(lambda x: [word for word in x if not word.isdigit()])

  #remove symbol from comments
  data[col] = data[col].apply(lambda x: [word for word in x if x!=""])

  #remove short word
  data[col] = data[col].apply(lambda x: [word for word in x if len(word)>2])

  #remove symbols
  data[col] = data[col].apply (lambda x: [re.sub(r"[^a-z]", "", word) for word in x])
  return
  """

'\n#create function to preprocess data\ndef preprocessor (data, col):\n  #Lower the lettercase\n  data[col] = data[col].str.lower()\n\n  #Remove stop words\n  stop_words = set(stopwords.words("english"))\n  data[col] = data[col].apply(lambda x: " ".join([word for word in str(x).split() if word not in (stop_words)]))\n\n  #Tokenize the word\n  data[col] = data[col].apply(word_tokenize)\n\n  #Remove numbers\n  data[col] = data[col].apply(lambda x: [word for word in x if not word.isdigit()])\n\n  #remove symbol from comments\n  data[col] = data[col].apply(lambda x: [word for word in x if x!=""])\n\n  #remove short word\n  data[col] = data[col].apply(lambda x: [word for word in x if len(word)>2])\n\n  #remove symbols\n  data[col] = data[col].apply (lambda x: [re.sub(r"[^a-z]", "", word) for word in x])\n  return\n  '

remove_freq_words function: This function removes top X percent of frequent words from the column.

In [5]:
"""
def remove_freq_words(data, col, percent):
  #Flatten the list of tokens
  all_tokens = [token for sublist in data[col] for token in sublist]

#calculate word frequencies
  word_freq = Counter(all_tokens)

#Convert it to DF
  word_freq_df = pd.DataFrame(word_freq.items(), columns = ["word","freq"])

#Identify the to 5% most frequent words
  top_5_percent = word_freq_df.nlargest(int(len(word_freq_df)*percent), "freq")["word"]

  filtered_data = []
  for sentence in data[col]:
    filtered_sentence = [word for word in sentence if word not in top_5_percent.values]
    filtered_data.append(" ".join(filtered_sentence))

  print(filtered_data)
  """

'\ndef remove_freq_words(data, col, percent):\n  #Flatten the list of tokens\n  all_tokens = [token for sublist in data[col] for token in sublist]\n\n#calculate word frequencies\n  word_freq = Counter(all_tokens)\n\n#Convert it to DF\n  word_freq_df = pd.DataFrame(word_freq.items(), columns = ["word","freq"])\n\n#Identify the to 5% most frequent words\n  top_5_percent = word_freq_df.nlargest(int(len(word_freq_df)*percent), "freq")["word"]\n\n  filtered_data = []\n  for sentence in data[col]:\n    filtered_sentence = [word for word in sentence if word not in top_5_percent.values]\n    filtered_data.append(" ".join(filtered_sentence))\n\n  print(filtered_data)\n  '

    Splits the input text into chunks that do not exceed max_chunk_size characters.
    The splitting is based on sentence boundaries.

    Parameters:
        text (str): The full text to be chunked.
        max_chunk_size (int): Maximum number of characters per chunk.

    Returns:
        List[str]: A list of text chunks.

In [67]:
def chunk_text(text, max_chunk_size=500):

    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = ""
    for sentence in sentences:
        if len(current_chunk) + len(sentence) + 1 > max_chunk_size:
            if current_chunk:
                chunks.append(current_chunk.strip())
                current_chunk = sentence + " "
            else:
                # In case a single sentence exceeds max_chunk_size.
                chunks.append(sentence.strip())
                current_chunk = ""
        else:
            current_chunk += sentence + " "
    if current_chunk.strip():
        chunks.append(current_chunk.strip())
    return chunks

## **Data**

JP Morgan QA section

In [49]:
#Obtaining management discussion / git bash
!git clone https://github.com/sheldonkemper/bank_of_england.git
!git switch Preprocessing
%cd bank_of_england/data/processed
%ls

#Defining qa_data
qa_data = pd.read_csv("qa_section.csv")


Cloning into 'bank_of_england'...
remote: Enumerating objects: 638, done.
remote: Counting objects: 100% (181/181), done.
remote: Compressing objects: 100% (132/132), done.
remote: Total 638 (delta 101), reused 51 (delta 47), pack-reused 457 (from 2)
Receiving objects: 100% (638/638), 4.27 MiB | 24.73 MiB/s, done.
Resolving deltas: 100% (274/274), done.
Already on 'Preprocessing'
Your branch is up to date with 'origin/Preprocessing'.
/content/bank_of_england/data/processed/bank_of_england/data/processed
management_discussion.csv  qa_section.csv  santander_management_discussion.csv


In [7]:
!git switch Preprocessing
%ls


Branch 'Preprocessing' set up to track remote branch 'Preprocessing' from 'origin'.
Switched to a new branch 'Preprocessing'
management_discussion.csv  qa_section.csv  santander_management_discussion.csv


In [12]:
%cd bank_of_england/notebooks/
%ls

/content/bank_of_england/notebooks
 import/  'import '/   library/   modelling/   processed/


In [13]:
from library import quant as qu

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [50]:
qa_data.head()

,speaker,marker,job_title,utterance,filename,financial_quarter,call_date
0,Jeremy Barnum,A,"Chief Financial Officer, JPMorganChase","Yeah. I think the conventional wisdom on QT, a...",4q24-earnings-transcript.pdf,4Q24,2025-01-15
1,Mike Mayo,Q,"Analyst, Wells Fargo Securities LLC","So, you'll stay around maybe for a few more ye...",4q24-earnings-transcript.pdf,4Q24,2025-01-15
2,Mike Mayo,Q,"Analyst, Wells Fargo Securities LLC",All right. Thank you.,4q24-earnings-transcript.pdf,4Q24,2025-01-15
3,Operator,NaN,NaN,Thank you. Our next question comes from Jim Mi...,4q24-earnings-transcript.pdf,4Q24,2025-01-15
4,Jim Mitchell,Q,"Analyst, Seaport Global Securities LLC","Hey. Good morning. Maybe just on regulation, w...",4q24-earnings-transcript.pdf,4Q24,2025-01-15


In [15]:
#Checking the type of data
qa_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 739 entries, 0 to 738
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   speaker            739 non-null    object
 1   marker             637 non-null    object
 2   job_title          636 non-null    object
 3   utterance          738 non-null    object
 4   filename           739 non-null    object
 5   financial_quarter  739 non-null    object
 6   call_date          739 non-null    object
dtypes: object(7)
memory usage: 40.5+ KB


In [51]:
qu.preprocessor(qa_data, "utterance")

In [52]:
qa_data.head()

,speaker,marker,job_title,utterance,filename,financial_quarter,call_date
0,Jeremy Barnum,A,"Chief Financial Officer, JPMorganChase","[yeah, think, convent, wisdom, pretend, add, c...",4q24-earnings-transcript.pdf,4Q24,2025-01-15
1,Mike Mayo,Q,"Analyst, Wells Fargo Securities LLC","[stay, around, mayb, year, base, case, right, ...",4q24-earnings-transcript.pdf,4Q24,2025-01-15
2,Mike Mayo,Q,"Analyst, Wells Fargo Securities LLC","[right, thank, you]",4q24-earnings-transcript.pdf,4Q24,2025-01-15
3,Operator,NaN,NaN,"[thank, you, next, question, come, jim, mitche...",4q24-earnings-transcript.pdf,4Q24,2025-01-15
4,Jim Mitchell,Q,"Analyst, Seaport Global Securities LLC","[hey, good, morn, mayb, regul, new, administr,...",4q24-earnings-transcript.pdf,4Q24,2025-01-15


In [18]:
# Remove Operator
qa_data = qa_data[qa_data["speaker"] != "Operator"]

#Filtering only on Answers
#qa_data = qa_data[qa_data["marker"]=="A"]

In [19]:
filtered_data = qu.remove_freq_words(qa_data, "utterance", 0.05)

Santander management discussion

In [30]:
%cd ..
%cd content/bank_of_england/data/processed/
%ls

/
/content/bank_of_england/data/processed
management_discussion.csv  qa_section.csv  santander_management_discussion.csv


In [70]:
#defining santader dataframe
santander_df=pd.read_csv("santander_management_discussion.csv")

In [71]:
#adding headers on each column
santander_df.columns = ["file_name", "transcript","quarter","header"]

#dropping header
santander_df=santander_df.drop(columns=["header"])

In [92]:
santander_df.head()

,file_name,transcript,quarter
0,video_2024_Q1_1,"Good morning, everybody. And welcome to Banco...",2024 Q1
1,video_2024_Q2_2,"Good morning, everybody. And welcome to Banco...",2024 Q2
2,video_2024_Q3_3,I'm going to start with the presentation. Ple...,2024 Q3
3,video_2024_Q4_4,"Thank you for joining, Santander's 2024 Resul...",2024 Q4
4,video_2023_Q1_5,Good morning everybody and welcome to Banco S...,2023 Q1


In [122]:
#chuncking

file_name=[]
quarter_name=[]
chunk_total=[]

for t in range(len(santander_df["transcript"])):
  #print(santander_df["file_name"][t])

  chunks = chunk_text(santander_df["transcript"][t], max_chunk_size=500)
  #print(len(chunks))
  for u in range(len(chunks)):
     file_name.append(santander_df["file_name"][t])
     quarter_name.append(santander_df["quarter"][t])
  #print(len(chunks))
  #print("t",chunks[1])
  if chunks:
    dummy_df["file_name"][t]=santander_df["file_name"][t]
    print(len(chunks))
    chunk_total.append(chunks)
    #dummy_df["chunks"][t]=chunks[t]
  else:
    print(f"No chunks produced for {t['filename']}.")

print(len(file_name))
print(chunk_total)
flat_list = [item for sublist in chunk_total for item in sublist]
print(len(flat_list))
data={
    "file_name":file_name,"chunks":flat_list,"quarter":quarter_name
}
dummy_df = pd.DataFrame(data)

105
153
177
175
143
105
153
177
179
143
68
172
90
1840
[["Good morning, everybody. And welcome to Banco Santander's conference call to discuss our financial results for the first quarter of 2024. Just as a reminder, both the results report and presentation we will be following today are available to you on our website. Let me just highlight that this is the first full quarter in which we are publishing results with our global businesses as primary reporting, aligned with the way we are managed as we announced late last year.", 'Secondary reporting will be what used to be primary until 2023, mainly country reporting, European DCB, etc. As we will explain later, changes versus previous periods are done on constant euros with the exception of Argentina which is presented in current euros to avoid distortions. All information will be as always available in the Excel sheet and reports that is published in our website.', 'But now to the presentation, I am joined here today by our CEO, Mr. Ek

In [98]:
print(file_name)

['video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024_Q1_1', 'video_2024

In [123]:
dummy_df.head()

,file_name,chunks,quarter
0,video_2024_Q1_1,"Good morning, everybody. And welcome to Banco ...",2024 Q1
1,video_2024_Q1_1,Secondary reporting will be what used to be pr...,2024 Q1
2,video_2024_Q1_1,"But now to the presentation, I am joined here ...",2024 Q1
3,video_2024_Q1_1,"First, let me share with you what we will focu...",2024 Q1
4,video_2024_Q1_1,"That's an 11% increase versus Q1, 23. 9% in co...",2024 Q1


In [61]:
qu.preprocessor(santander_df,"transcript")

In [62]:
santander_df.head()

,file_name,transcript,quarter
0,video_2024_Q1_1,"[good, morn, everybodi, welcom, banco, santand...",2024 Q1
1,video_2024_Q2_2,"[good, morn, everybodi, welcom, banco, santand...",2024 Q2
2,video_2024_Q3_3,"[go, start, present, plean, everyon, welcom, t...",2024 Q3
3,video_2024_Q4_4,"[thank, join, santand, result, present, raul, ...",2024 Q4
4,video_2023_Q1_5,"[good, morn, everybodi, welcom, banco, santand...",2023 Q1


# **Export the output as a csv file**

In [21]:
#export preprocessed data for Modeling
preprocessed_qa_csv_path = "/content/sample_data/test.csv"

qa_data.to_csv(preprocessed_qa_csv_path, index=False)

In [124]:
preprocessed_qa_csv_path2 = "/content/sample_data/chunk_text.csv"
dummy_df.to_csv(preprocessed_qa_csv_path2, index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **Modeling**

In [22]:
# Define a function to reset the session.
def reset_session():
    tf.keras.backend.clear_session()
    np.random.seed(42)
    random.seed(42)
    tf.random.set_seed(42)
#reset_session()


#embedding model to numarise the text
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

#umap model to reduce dimention
umap_model = UMAP(n_neighbors=20, min_dist=0.1)

#representation mode to adjust topic or keywords. diversity is adjustable depending on the result
representation_model = MaximalMarginalRelevance(diversity=0.2)

#BERTopic model: add above models in the parameters
bertModel = BERTopic(embedding_model=embedding_model, umap_model=umap_model, representation_model=representation_model, verbose=True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [23]:
#Fitting data into bertModel
bertModel.fit(filtered_data)
topic, probabilities = bertModel.transform(filtered_data)

2025-02-10 14:15:28,169 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/20 [00:00<?, ?it/s]

2025-02-10 14:15:45,870 - BERTopic - Embedding - Completed ✓
2025-02-10 14:15:45,875 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-02-10 14:16:03,664 - BERTopic - Dimensionality - Completed ✓
2025-02-10 14:16:03,668 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-02-10 14:16:03,712 - BERTopic - Cluster - Completed ✓
2025-02-10 14:16:03,721 - BERTopic - Representation - Extracting topics from clusters using representation models.
2025-02-10 14:16:05,356 - BERTopic - Representation - Completed ✓


Batches:   0%|          | 0/20 [00:00<?, ?it/s]

2025-02-10 14:16:24,270 - BERTopic - Dimensionality - Reducing dimensionality of input embeddings.
2025-02-10 14:16:24,275 - BERTopic - Dimensionality - Completed ✓
2025-02-10 14:16:24,278 - BERTopic - Clustering - Approximating new points with `hdbscan_model`
2025-02-10 14:16:24,301 - BERTopic - Cluster - Completed ✓


# **Result**

In [24]:
bertModel.get_topic_freq().head(10)

,Topic,Count
0,0,423
1,1,119
2,2,37
6,3,19
4,-1,16
3,4,14
5,5,12


In [25]:
bertModel.get_topic(0)

[('cut', 0.012603456342721694),
 ('deploy', 0.012603456342721694),
 ('card', 0.012399107137942058),
 ('fed', 0.012399107137942058),
 ('spend', 0.011985779104187728),
 ('margin', 0.011985779104187728),
 ('expens', 0.011916108040153842),
 ('let', 0.011507503617267635),
 ('share', 0.011353642074767368),
 ('economi', 0.011139525734749583)]

In [26]:
import plotly.io as pio

fig = bertModel.visualize_barchart(top_n_topics=10, n_words=5)
fig.update_layout(
    autosize=False,
    width=1000,
    height=800,
    margin=dict(l=50, r=50, t=100, b=50),
    font=dict(size=12),
    title=dict(
        text="Top 10 Topics and Their Key Words",
        font=dict(size=16),
        x=0.5,
        y=0.98,
        xanchor="center",
        yanchor="top"
    )
)

fig.show()